## Objectives 

- Familiarization with smart contarcts 
- Step through OPCODE level execution of a transaction
- Familiarize and set up web3 library and wallet APIs 
- Interacting with an ERC20 token
- Interacting with a contract using Etherscan
- Importing ERC20 tokens on Metamask

### Contracts we will be using
- USTUSD: `0xc83B0efA5B3F13851DfA11de72EF6AFeF026730c`
- USTETH: `0x4E22e9951770b98d4f3B2BA6647f0f40A15A4057`
- Faucet: `0x4Cd26B8a999582727789E8236789125D8a9022c5`

## Walkthrough smart contract

We have deployed smart contracts for the ERC20 tokens on the Sepolia testnet. The contracts can be found on etherscan here: 
- [USTUSD](https://sepolia.etherscan.io/address/0xc83B0efA5B3F13851DfA11de72EF6AFeF026730c)
- [USTETH](https://sepolia.etherscan.io/address/0x4E22e9951770b98d4f3B2BA6647f0f40A15A4057)

We will pretend that the `HKUSTGZ USD` token has a value of 1 USD, and the `HKUSTGZ ETH` token value is decided by the market (i.e. participants such as you buying/selling these tokens).

Let us now go through the contract and describe what each ```function``` does.





### What does ```transfer``` and ```transferFrom``` do ? Which of them needs you to run ```approve``` before it executes correctly ?

### What does ```balanceOf``` and ```totalSupply``` do?

1. **totalSupply()**: Returns the total number of tokens in circulation (read-only, no gas cost).
2. **balanceOf(address account)**: Returns the token balance of the specified account (read-only, no gas cost).
3. **transfer(address recipient, uint256 amount)**: Transfers `amount` tokens from the caller’s account to `recipient` (no approval needed).
4. **transferFrom(address sender, address recipient, uint256 amount)**: Transfers `amount` tokens from `sender` to `recipient` by a third-party caller (**requires prior approval from sender to caller**).
5. **approve(address spender, uint256 amount)**: Allows `spender` to withdraw up to `amount` tokens from the caller’s account (prerequisite for `transferFrom`).

You can have try by calling these functions on Etherscan under: `Contract` -> `Read Contract` or `Write Contract`.
You'd better connect your wallet first by clicking `Connect to Web3` .
- https://sepolia.etherscan.io/address/0xc83B0efA5B3F13851DfA11de72EF6AFeF026730c#readContract
- https://sepolia.etherscan.io/address/0x4E22e9951770b98d4f3B2BA6647f0f40A15A4057#readContract
- https://sepolia.etherscan.io/address/0x4Cd26B8a999582727789E8236789125D8a9022c5#readContract




### What events can be emitted by the contract ? [What is their use](https://consensys.net/blog/developers/guide-to-events-and-logs-in-ethereum-smart-contracts/) ? 

# Requesting tokens

To request the tokens we shall be using in the future for some in-class labs, we are going to use the ```web3``` library in python. The installation instructions can be found [here](https://web3py.readthedocs.io/en/v5/quickstart.html). We will view our transactions on the [Sepolia testnet etherscan](https://sepolia.etherscan.io/). 

We now send a transaction that uses this API to request some of the Lab ERC20 tokens from the faucet contract. The faucet allows each registered wallet to redeem a fixed number of tokens per day. The code used to do that has to be written below.

### Setup Web3 API
First, we connect your wallet to a Web3 API - this is a service that connects you to the Ethereum blockchain via a simple URL. 

To do that, first create a free account on [Infura](https://app.infura.io/register) (using any email ID).

Once you log in, you would see your Infura API key under the project name you had given. Use that key in the following code.

<img src="./figs/fig1.png" width="70%" />

In [ ]:
# Import web3 library
from web3 import Web3

# Setup variables
infura_key = 'your-infura-project-id'  # Replace with your Infura Project ID or set in .env
wallet_address = 'your-wallet-address'  # Replace with your wallet address or set in .env
wallet_private_key = 'your-private-key'  # Replace with your private key or set in .env

w3 = Web3(Web3.HTTPProvider(f'https://sepolia.infura.io/v3/{infura_key}'))

print(w3.eth.block_number) #Check if the notebook is connected to the Infura node
print(w3.__dict__)

10110686
{'manager': <web3.manager.RequestManager object at 0x000001FAF4201720>, 'codec': <eth_abi.codec.ABICodec object at 0x000001FAF4201630>, 'eth': <web3.eth.eth.Eth object at 0x000001FAF4200460>, 'net': <web3.net.Net object at 0x000001FAF4200D00>, 'geth': <web3.geth.Geth object at 0x000001FAF4200B50>, 'tracing': <web3.tracing.Tracing object at 0x000001FAF4200A30>, 'testing': <web3.testing.Testing object at 0x000001FAF4200AC0>, '_ens': <web3._utils.empty.Empty object at 0x000001FAF3829210>}


### Setup your ETH account details

In [ ]:
 # Wallet details
public_address = Web3.to_checksum_address(wallet_address) # see metamask for this
private_key = wallet_private_key # see metamask for this
print(public_address)

0x93753aa12E364f76a2bE0C22F1a4e4A443E715f1


### Set up a contract instance
Now, we send a request transaction to the faucet, signed by your private key.

In [ ]:
# Define the USTFaucet contract to connect to
faucet_address = Web3.to_checksum_address('0x4Cd26B8a999582727789E8236789125D8a9022c5')

# You can get the ABI from Etherscan: https://sepolia.etherscan.io/address/0x4Cd26B8a999582727789E8236789125D8a9022c5#code
abi_contract = 'get-the-ABI-from-Etherscan-and-paste-here'  # Replace this string with the actual ABI JSON
contract = w3.eth.contract(faucet_address, abi=abi_contract)

print(contract.functions.MAX_DAILY_REQUESTS().call())  # Example of calling a read function from the contract

0x633C62F02037Dc718ee298E127194C00ab8dB803


### Create transaction data

In [ ]:
### Create transaction data
txn = contract.functions.requestTokens().build_transaction(
    {   
        # nonce is required to prevent replay attacks
        'nonce':w3.eth.get_transaction_count(public_address),
        'from':public_address, # `from` is the address sending the transaction
    }
)

print(txn)

{'value': 0, 'gas': 68676, 'maxPriorityFeePerGas': 1439189, 'maxFeePerGas': 2051012993, 'chainId': 11155111, 'nonce': 93, 'from': '0x93753aa12E364f76a2bE0C22F1a4e4A443E715f1', 'to': '0x2706B620345A1bcAD9D8e560D46Fb503e9e3c1B4', 'data': '0x580f3904000000000000000000000000633c62f02037dc718ee298e127194c00ab8db803'}


### Sign and post transaction
You should see the hash of your transaction printed.

In [ ]:
# Post transaction
signed_txn = w3.eth.account.sign_transaction(txn, private_key) # Sign the txn with your private key 

raw_transaction = signed_txn.rawTransaction if hasattr(signed_txn, 'rawTransaction') else signed_txn.raw_transaction

txn_hash = w3.eth.send_raw_transaction(raw_transaction) # Send it onto the chain
print(txn_hash.hex())  # Print the transaction hash
# You can check the transaction status on Sepolia Etherscan

d1d4194965e75e193934547ddf0231e78230110d7127ccd87b2cc322f9228b0e


## Alternate: Request tokens through Etherscan

Verified contracts on Etherscan allow you to interact with functions on the contract.

Open the [USTFaucet](https://sepolia.etherscan.io/address/0x4Cd26B8a999582727789E8236789125D8a9022c5#code) contract by providing it's address. You can now access the contract's read and write functions.

Go to the ``write tab``, connect your metamask wallet and call the ``requestTokens`` function by providing your wallet's public address and the HUSD token's contract address. Confirm the transaciton on the metamask prompt.

# Add to MetaMask

- The token balances can now be tracked via MetaMask using the contract addresses of each token. 
- Use the "Import Tokens" link at the bottom of your MetaMask. 
- Enter the smart contract address for an ERC20 tokens we would be using in the class.
- Do this for each of the tokens - HUSD and PEARL.